In [378]:
#### This script was used at the start of the project but it was not efficient as every possible kmer was included
####even if it was not found in any sequence. The storage cost were higher

## Select a k-mer length and it will create a Matrix of the k-mer counts for all sequences in the file
## With big numbers of K, the matrix is totally inefficient as it has a lot of empty columns
## This script should be changed to the one that creates one dictionary for each cluster
## THEREFORE, DONT USE THIS SCRIPT ANYMORE. JUST SAVED FOR FUTURE REFERENCE

In [379]:
import numpy as np

In [380]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial


In [381]:
inputFile = open ("DNASequencesFasta.txt")

In [382]:
##The file is read and the number of clusters is counted
numberOfClusters=0
for line in inputFile:
    if line[0:1]==">":
        numberOfClusters+=1
inputFile.seek(0)#Goes back to the start of the file for the next file handler

0

In [383]:
sequence=""
clusterNumber=0
kmerLength=8
matrix=np.zeros((numberOfClusters,4**kmerLength))
wrongBasesList=["N","S","W","Y","R","K","D","M","V","H","B"] ##This bases sometimes appear in the middle of the DNA
flag=False #So the function is not called in the first iteration

In [384]:
def obtainingKmers (sequence,kmerLength):
    for i in range (0,len(sequence),kmerLength):
        kmer = sequence[i:i+kmerLength]
        ##If the last kmer is shorter or if a kmer has a wrong base, it is ignored
        if len(kmer)!=kmerLength or any (wrongBase in kmer for wrongBase in wrongBasesList):
            continue
        column=from4BasedToColumnInTheMatrix(fromKmerTo4BasedValue(kmer))##Column of the matrix for that kmer
        matrix[clusterNumber, column]+=1 ##Increases that cell by 1 count

In [385]:
def fromKmerTo4BasedValue(kmer): ##Converts the kmer into a quaternary number
    lettersValue={"A":"0","C":"1","G":"2","T":"3"}
    value4Based=""
    for nucleotide in kmer:
        value4Based+=lettersValue[nucleotide]
    return value4Based

In [386]:
def from4BasedToColumnInTheMatrix(value4Based): ##Takes a quaternary number and says the column of the matrix related to it
    exponent=len(value4Based)-1
    outputColumn=0
    for value in value4Based:
        outputColumn+=int(value)*(4**exponent)
        exponent-=1
    return outputColumn

In [387]:
##The first time the flag is false so the function is not called yet.
##From that point on, when we find a new cluster, the method is called for the previous cluster
##The last line outside the loop calls the method for the last cluster
for line in inputFile:
    line=line.strip("\n")
    if line[0:1]==">":
        if flag:#Calls the function for the previous cluster
            obtainingKmers(sequence,kmerLength)
            clusterNumber+=1
        else:
            flag = True
        sequence=""
    else:
        sequence+=line
obtainingKmers(sequence,kmerLength)

In [388]:
inputFile.close()

In [389]:
matrix.view()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [390]:
np.savetxt('trialMatrix.csv',matrix,delimiter=',',fmt="%d")